# Numpy and arrays
In class we discussed the ins and outs of using modules and discussed one package `numpy` in detail. Let's do some work with `numpy.ndarray`. 

Question 1:  Construct a class `rowlabels` that puts labels on axis 0 of an `np.ndarray`after constructing the `np.ndarray` normally. 

* Its constructor takes three arguments: `self`, the input `data` to `ndarray.__init__` and a flat list `labels` of the axis zero labels that should have the same length as axis zero of `data`. 
  * It should raise an exception if the labels list and data list are not the same length.
  * It should raise an exception if the labels list has duplicate labels. 
* The method `rowlabels.bylabel(label)` should look up the axis zero entry for that label and return it. 
  * It should raise an exception if the given label was not defined in the constructor. 

e.g., after the code 
        
        x = rowlabels([1,2,3], ['foo', 'bar', 'cat'])
* `x.bylabel('foo')` should return `1` while 
* `x.bylabel('bar')` should return `2`.
* `x.bylabel('dog')` should raise an exception.

Hints: 
* This is not a subclass. `np.nparray` is difficult to subclass. Instead, embed an instance of `np.nparray` inside `rowlabels`. 
* For the purposes of these exercises, raise a generic `Exception`. I realize this is poor style, but I would like you to finish this in one lab period:) 
* I used a regular `dict` to remember which label goes with which row offset. In the above example, this `dict` is `{'foo': 0, 'bar': 1, 'cat': 2}`. 

*Note that you are building a class that is very similar to a Pandas Series.* 

In [104]:
import numpy as np

class rowlabels: 
    
    def __init__(self, data, labels):
        # BEGIN SOLUTION

        # Exceptions if length doesn't match
        if (len(labels) != len(data)):
            raise exception("data length doesn't match label length")

        # Exception if duplicates found
        set_labels = set(labels)
        if (len(labels) != len(set_labels)):
            raise exception("duplicates found in labels")

        # Ensure data entries are stored as an np.array
        self.data = np.array(data)
        self.labels = labels

        # Use a map (dict) to store and retrieve values
        self.label_map = {label: value for value, label in enumerate(labels)} 
    
        # END SOLUTION
        
    def bylabel(self, label):
        # Exception if the label is not found
        if label not in self.label_map:
            raise Exception(f"Label '{label}' not found.")
        # BEGIN SOLUTION
        #Call by index to match the assert below which seeks a numpy array not just the int index
        search_indx = self.label_map[label]
        return self.data[search_indx]
        # END SOLUTION

In [79]:
stuff = rowlabels([1, 2, 3], ['cat', 'dog', 'bat'])
assert stuff.bylabel('cat') == 1
assert stuff.bylabel('dog') == 2
assert stuff.bylabel('bat') == 3

In [80]:
from functools import reduce
stuff = rowlabels([[1,2]],['foo'])
assert isinstance(stuff.bylabel('foo'), np.ndarray)
assert reduce(lambda a,b: a and b, stuff.bylabel('foo') == np.array([1,2]))

In [81]:
stuff = rowlabels([[1,2]],['foo'])
try: 
    stuff.bylabel('dog')
    print("fetching non-existent label worked and should not.")
except Exception: 
    pass

In [82]:
try: 
    x = rowlabels([1,2,3], ['cat', 'dog'])
    print("able to create a labeledarray with a different number of labels than rows.")
except Exception: 
    pass

In [83]:
try: 
    x = rowlabels([1,2,3], ['cat', 'dog', 'cat'])
    print("able to create a labeledarray with duplicate labels!")
except Exception: 
    pass

Question 2: Often, data is not organized so nicely. Create a subclass `rawrowlabels` of `rowlabels` that takes a different form of initializer: a list of rows each with structure `['label', ...row structure...]` where `'label'` is the name of the row and `...row structure...` is replaced by row data of parallel structure for each row. E.g., 

* `x = rawrowlabels([['foo', 1, 2], ['bar', 1, 3]])` and then
* `x.bylabel('foo'))` 

would return `np.array([1,2])`

Hints: 
* Look up and use array slice syntax `a[1:]`.
* You only have to define the new constructor. Everything else is defined in the previous problem. 

In [106]:
class rawrowlabels(rowlabels):  
    def __init__(self, data): 
        # BEGIN SOLUTION
        labels = []
        row_data = []

        # New nparray for row_data
        # Iterate for each 'row', or embedded sub-list
        for list in data:
            labels.append(list[0])
            row_data.append(np.array(list[1:]))

        # Use the same init and get the map using these new values
        super().__init__(row_data, labels)
        # END SOLUTION

In [99]:
stuff = rawrowlabels([['cat', 1], ['dog', 2], ['rat', 3]])
assert stuff.bylabel('cat')[0] == 1
assert stuff.bylabel('dog')[0] == 2
assert stuff.bylabel('rat')[0] == 3

In [100]:
stuff = rawrowlabels([['cat', 1, 2], ['dog', 1, 3], ['horse', 2, 4]])
from functools import reduce
assert isinstance(stuff.bylabel('dog'), np.ndarray)
assert reduce(lambda a,b: a and b, stuff.bylabel('dog') == np.array([1,3]))

In [101]:
stuff = rawrowlabels([['cat', 1, 2], ['dog', 1, 3], ['horse', 2, 4]])
from functools import reduce
assert isinstance(stuff.bylabel('cat'), np.ndarray)
assert reduce(lambda a,b: a and b, stuff.bylabel('cat') == np.array([1,2]))

In [102]:
stuff = rawrowlabels([['cat', 1, 2], ['dog', 1, 3], ['horse', 2, 4]])
from functools import reduce
assert isinstance(stuff.bylabel('horse'), np.ndarray)
assert reduce(lambda a,b: a and b, stuff.bylabel('horse') == np.array([2,4]))

In [103]:
try: 
    x = rawrowlabels([['cat', 1],['dog', 2],['cat', 3]])
    print("able to create a labeledarray with duplicate labels!")
except Exception: 
    pass